<a href="https://colab.research.google.com/github/sayakpaul/Generalized-ODIN-TF/blob/main/Calculate_Epsilon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
# Retrieve the pre-trained ResNet20 model.
# Refer to this notebook for details: https://git.io/JOJuL.
!gdown --id 1-6Xp7__VMJeMBtGMByI7_GYVeIIv-1Ay -O odin_rn_model.tar.gz
!tar xf odin_rn_model.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1-6Xp7__VMJeMBtGMByI7_GYVeIIv-1Ay
To: /content/odin_rn_model.tar.gz
6.62MB [00:00, 40.2MB/s]


In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

## Load the pre-trained model

In [2]:
model = tf.keras.models.load_model("odin_rn_model")
print(f"Pre-trained model loaded with {model.count_params()/1e6} M parameters.")

Pre-trained model loaded with 0.572223 M parameters.


## Load the CIFAR-10 dataset

In [3]:
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(f"Total test examples: {len(x_test)}")

Total test examples: 10000


## Define constants

In [4]:
BATCH_SIZE = 128
AUTO = tf.data.AUTOTUNE
SAMPLES_PERTURB = 1000

## Prepare data loaders

In [5]:
perturb_ds = tf.convert_to_tensor(x_test[:SAMPLES_PERTURB].astype("float32"))
ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = ds.skip(SAMPLES_PERTURB).take(len(x_test) - SAMPLES_PERTURB).batch(BATCH_SIZE)

## Calculating the perturbation magnitude ($\epsilon^{*}$)

From the paper: 

> In our method, we search for the $\epsilon^{*}$ which maximizes the score $S(x)$ with only the in-distribution validation dataset $D_{\text {in }}^{\text {val }}$:

$$\epsilon^{*}=\underset{\varepsilon}{\arg \max } \sum_{x \in D_{i n}^{\text {val }}} S({\boldsymbol{x}})$$

$S(x)$ is given by:

>  For out-of-distribution detection, we use the scoring function:
<center>
$S_{DeConf}(\boldsymbol{x})=\max _{i} h_{i}(\boldsymbol{x})$ or $g(\boldsymbol{x})$
</center>

Perturbation of an input image is realized using the equation below:

$$
\hat{\boldsymbol{x}}=\boldsymbol{x}-\epsilon \operatorname{sign}\left(-\nabla_{\boldsymbol{x}} S(\boldsymbol{x})\right)
$$

In [6]:
# Let's define our model to obtain scores.
scorer = tf.keras.Model(model.input, model.layers[-3].output)

In [8]:
# We can take a batch of data and obtain the OOD scores like so.
sample_scores = scorer.predict(test_ds.take(1))
sample_scores[:5], sample_scores.shape

(array([[ -7.3858304,  -4.398713 ,   2.3269448,   8.260478 ,   0.6237583,
           6.951867 ,   5.303116 ,  -1.433213 ,  -5.521435 ,  -4.7768483],
        [ 11.8561945,   3.8889992,   0.7458444,  -3.396165 ,  -3.631293 ,
          -8.730821 ,  -8.521949 ,  -5.3349705,  10.599314 ,   3.2314248],
        [  1.4345236,   4.5795317,  -4.922901 ,  -1.4036714,  -5.604335 ,
          -3.132948 ,  -5.479171 ,  -3.4059682,  15.415438 ,   5.6661053],
        [ -6.804879 ,  -8.259593 ,   1.7251043,   4.7501116,  16.202501 ,
           8.437861 ,  -2.1132357,   5.5491834, -11.250345 ,  -5.944642 ],
        [  5.1940913,   8.496859 ,  -6.5266933,  -2.5342746,  -9.523632 ,
          -6.7520833,  -9.587466 ,  -9.912749 ,  28.651361 ,   6.293258 ]],
       dtype=float32), (128, 10))

In [9]:
# Grid as defined in Section 3.2.
epsilon_grid = [0.0025, 0.005, 0.01, 0.02, 0.04, 0.08]

In [17]:
def perturb_images(model, epsilon):
    test_ds_var = tf.Variable(perturb_ds, trainable=True)
    
    with tf.GradientTape() as tape:
        # Calculate the scores.
        tape.watch(test_ds_var)
        logits = model(test_ds_var, training=False)
        scores = tf.reduce_max(logits, axis=1)

    # Calculate the gradients of the scores with respect to the inputs.
    # (Notice the sign.)
    gradients = -tape.gradient(scores, test_ds_var)

    # Perturb the inputs and derive new mean score.
    test_ds_var.assign_sub(epsilon * tf.sign(gradients))
    static_tensor = tf.convert_to_tensor(test_ds_var)
    static_tensor = tf.clip_by_value(static_tensor, 0., 255.)
    new_mean_score = tf.reduce_mean(model.predict(static_tensor))

    return new_mean_score

In [18]:
# Derive the perturbation magnitude. 
mean_scores = []

for epsilon in epsilon_grid:
    mean_scores.append(perturb_images(scorer, epsilon).numpy())

epsilon_selected = epsilon_grid[np.argmax(mean_scores)]
print(f"Epsilon: {epsilon_selected}")

Epsilon: 0.08
